## 1. import data

In [12]:
import pandas as pd

df = pd.read_csv("E:/class_predict/50k_data_output_processed.tsv", sep='\t')
df.head()

,reactants,products,class,split,Reaction
0,CC1(CO[Si](C)(C)C(C)(C)C)Cc2nnc(-c3ccc(-c4ccc(...,CC1(CO)Cc2nnc(-c3ccc(-c4ccc(F)cc4)cc3C(F)(F)F)...,RX_6,test,CC1(CO[Si](C)(C)C(C)(C)C)Cc2nnc(-c3ccc(-c4ccc(...
1,CCCCOCCOc1ccc(-c2ccc3c(c2)C=C(C(=O)Nc2ccc([C@H...,CCCCOCCOc1ccc(-c2ccc3c(c2)C=C(C(=O)Nc2ccc([C@H...,RX_6,test,CCCCOCCOc1ccc(-c2ccc3c(c2)C=C(C(=O)Nc2ccc([C@H...
2,CC(C)(C)OC(=O)NC(CC(=O)N1Cc2ccccc2-n2nnc(-c3cc...,N[C@@H](CC(=O)N1Cc2ccccc2-n2nnc(-c3ccc(F)cc3)c...,RX_6,test,CC(C)(C)OC(=O)NC(CC(=O)N1Cc2ccccc2-n2nnc(-c3cc...
3,CCOC(=O)c1cn(CC)c2c(F)c(OCc3ccc(OC)cc3)c(OCc3c...,CCn1cc(C(=O)O)c(=O)c2cc(OCc3ccc(OC)cc3)c(OCc3c...,RX_6,test,CCOC(=O)c1cn(CC)c2c(F)c(OCc3ccc(OC)cc3)c(OCc3c...
4,COC(=O)c1cnc(CCc2c(-c3ccccn3)noc2C)s1,Cc1onc(-c2ccccn2)c1CCc1ncc(C(=O)O)s1,RX_6,test,COC(=O)c1cnc(CCc2c(-c3ccccn3)noc2C)s1>>Cc1onc(...


## 2. import rxnfp

In [13]:
import numpy as np
fps_ft_50k_class = np.load("E:/class_predict/50k_class_processed_rxnfp.npz")['fps']

In [14]:
fps_ft_50k_class.shape

(50016, 256)

## 3. data split

In [15]:
help(np.unique)

Help on function unique in module numpy:

unique(ar, return_index=False, return_inverse=False, return_counts=False, axis=None)
    Find the unique elements of an array.
    
    Returns the sorted unique elements of an array. There are three optional
    outputs in addition to the unique elements:
    
    * the indices of the input array that give the unique values
    * the indices of the unique array that reconstruct the input array
    * the number of times each unique value comes up in the input array
    
    Parameters
    ----------
    ar : array_like
        Input array. Unless `axis` is specified, this will be flattened if it
        is not already 1-D.
    return_index : bool, optional
        If True, also return the indices of `ar` (along the specified axis,
        if provided, or in the flattened array) that result in the unique array.
    return_inverse : bool, optional
        If True, also return the indices of the unique array (for the specified
        axis, if pro

In [16]:
np.unique(df["split"], return_counts=True)

(array(['test', 'train'], dtype=object), array([40072,  9944], dtype=int64))

In [17]:
df["split"]=="train"

0        False
1        False
2        False
3        False
4        False
         ...  
50011     True
50012    False
50013    False
50014    False
50015    False
Name: split, Length: 50016, dtype: bool

In [18]:
X_train = fps_ft_50k_class[df["split"]=="train"]

In [19]:
X_test = fps_ft_50k_class[df['split']=='test']

In [20]:
fps_ft_50k_class.shape

(50016, 256)

In [21]:
fps_ft_50k_label = df["class"].values #取出“rxn_class”的值（numpy.array）
fps_ft_50k_label.shape

(50016,)

In [22]:
y_train = fps_ft_50k_label[df["split"]=="train"]
y_train.shape

(9944,)

In [23]:
y_test = fps_ft_50k_label[df["split"]=="test"]
y_test.shape

(40072,)

In [24]:
y_test.shape[0], X_test.shape[0], y_test.shape, X_test.shape

(40072, 40072, (40072,), (40072, 256))

In [25]:
y_train.shape[0], X_train.shape[0], y_train.shape, X_train.shape

(9944, 9944, (9944,), (9944, 256))

## 4.training

In [26]:
from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# data = load_breast_cancer()
# X = data.data
# y= data.target

# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 90%|████████▉ | 26/29 [03:53<00:28,  9.64s/it]

[22:55:03] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 29/29 [04:26<00:00,  9.19s/it]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
LinearSVC                          0.97               0.94    None      0.97   
CalibratedClassifierCV             0.97               0.93    None      0.97   
PassiveAggressiveClassifier        0.97               0.93    None      0.97   
XGBClassifier                      0.97               0.92    None      0.97   
LGBMClassifier                     0.97               0.92    None      0.97   
KNeighborsClassifier               0.97               0.91    None      0.96   
ExtraTreesClassifier               0.97               0.91    None      0.97   
Perceptron                         0.96               0.91    None      0.96   
LinearDiscriminantAnalysis         0.96               0.90    None      0.96   
SVC                                0.96               0.90    None      0.96   
RandomForestClassifier             0.96 

In [27]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearSVC,0.97,0.94,None,0.97,5.00
CalibratedClassifierCV,0.97,0.93,None,0.97,17.53
PassiveAggressiveClassifier,0.97,0.93,None,0.97,1.46
XGBClassifier,0.97,0.92,None,0.97,23.11
LGBMClassifier,0.97,0.92,None,0.97,9.56
KNeighborsClassifier,0.97,0.91,None,0.96,36.13
ExtraTreesClassifier,0.97,0.91,None,0.97,3.02
Perceptron,0.96,0.91,None,0.96,1.31
LinearDiscriminantAnalysis,0.96,0.90,None,0.96,0.79


In [28]:
clf.models["PassiveAggressiveClassifier"].predict(X_test[0].reshape(-1,len(X_test[0])))

array(['RX_6'], dtype='<U5')

In [29]:
X_test

array([[-2.42642283, -0.41474456, -1.2058388 , ...,  0.04420783,
         0.23803483,  0.83875763],
       [-1.17005241,  0.70554012,  1.22783184, ..., -1.89423716,
        -0.3104338 ,  0.39099631],
       [-0.93451774,  1.07715988,  1.63014126, ..., -2.08379984,
        -0.18617198,  0.40866128],
       ...,
       [-1.37151849,  2.55586886, -1.3362323 , ..., -1.52197099,
        -0.79028052, -0.36866501],
       [-1.08525825,  2.83790255, -1.24917293, ..., -1.55910897,
        -0.89319587, -0.38679698],
       [ 0.55337906,  0.48286247, -1.49116206, ...,  0.93149149,
         0.92328787,  0.51479268]])

In [31]:
y_test[0]

'RX_6'

## 5. predict

In [33]:
import gzip
import pandas as pd
import numpy as np
from itertools import islice
from tqdm import tqdm, tqdm_notebook
from rxnfp.transformer_fingerprints import (
    RXNBERTFingerprintGenerator, get_default_model_and_tokenizer, generate_fingerprints
)

In [34]:
model, tokenizer = get_default_model_and_tokenizer('bert_ft_10k_25s')
ft_7k_rxnfp_generator = RXNBERTFingerprintGenerator(model, tokenizer)

# fps_ft_10k = generate_fingerprints(df.rxn.values.tolist(), ft_10k_rxnfp_generator, batch_size=8)
# np.savez_compressed('../data/fps_ft_10k', fps=fps_ft_10k)
# fps_ft_10k.shape

You passed along `num_labels=50` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
Some weights of the model checkpoint at d:\anaconda\envs\myenv\lib\site-packages\rxnfp\models\transformers\bert_ft_10k_25s were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
df_full = pd.read_csv("E:/class_predict/data/test/total_data_1010_processed.tsv", sep='\t')
ls=df_full['Reaction'].tolist()



fps_ft_7k_ = generate_fingerprints(df_full['Reaction'].tolist(), ft_7k_rxnfp_generator, batch_size=8)
np.savez_compressed("E:/class_predict/data/test/total_data_1010_processed_rxnfp.npz", fps=fps_ft_7k_)
fps_ft_7k_.shape

100%|██████████| 954/954 [03:14<00:00,  4.90it/s]


(7632, 256)

In [38]:
fps_ft_7k_.shape

(7632, 256)

In [40]:
clf.models["PassiveAggressiveClassifier"].predict(fps_ft_7k_[0].reshape(-1,len(fps_ft_7k_[0])))

array(['RX_9'], dtype='<U5')

In [46]:
for i in range(7632):  
    a = clf.models["PassiveAggressiveClassifier"].predict(fps_ft_7k_[i].reshape(-1,len(fps_ft_7k_[i])))
    print(i, a)

0 ['RX_9']
1 ['RX_1']
2 ['RX_9']
3 ['RX_3']
4 ['RX_6']
5 ['RX_3']
6 ['RX_4']
7 ['RX_7']
8 ['RX_9']
9 ['RX_9']
10 ['RX_4']
11 ['RX_6']
12 ['RX_3']
13 ['RX_7']
14 ['RX_9']
15 ['RX_3']
16 ['RX_3']
17 ['RX_8']
18 ['RX_9']
19 ['RX_3']
20 ['RX_7']
21 ['RX_6']
22 ['RX_7']
23 ['RX_7']
24 ['RX_9']
25 ['RX_3']
26 ['RX_8']
27 ['RX_3']
28 ['RX_3']
29 ['RX_7']
30 ['RX_3']
31 ['RX_4']
32 ['RX_3']
33 ['RX_3']
34 ['RX_7']
35 ['RX_7']
36 ['RX_9']
37 ['RX_3']
38 ['RX_9']
39 ['RX_3']
40 ['RX_6']
41 ['RX_7']
42 ['RX_7']
43 ['RX_9']
44 ['RX_3']
45 ['RX_9']
46 ['RX_2']
47 ['RX_7']
48 ['RX_8']
49 ['RX_7']
50 ['RX_3']
51 ['RX_8']
52 ['RX_3']
53 ['RX_5']
54 ['RX_7']
55 ['RX_7']
56 ['RX_9']
57 ['RX_1']
58 ['RX_1']
59 ['RX_1']
60 ['RX_1']
61 ['RX_3']
62 ['RX_7']
63 ['RX_3']
64 ['RX_9']
65 ['RX_2']
66 ['RX_7']
67 ['RX_8']
68 ['RX_3']
69 ['RX_1']
70 ['RX_3']
71 ['RX_3']
72 ['RX_7']
73 ['RX_6']
74 ['RX_3']
75 ['RX_6']
76 ['RX_6']
77 ['RX_3']
78 ['RX_3']
79 ['RX_3']
80 ['RX_3']
81 ['RX_6']
82 ['RX_3']
83 ['RX_1']
84